# Capstone Project - The Battle of Neighborhoods (Week 1-2)

## Business Problem 
#### Background
According to the main financial news firms, the Housing Market in Milan (Italy) has been facing for the last five years a growing trend in the households selling prices as well in the rental ones.
A further prices acceleration is expected in 2020 due to the "Brexit", which will probably result in a shift of real estate investments from London to the other UE cities.

#### Business Problem
In this scenario, many students who intend to pursue their university studies in Milan need to know more about the real estate market, in order to correctly choose the neighborhood where to live according to the available public services and venues and to the households buying or rental prices.
For this purpose it is possible to run machine learning models (clustering) in order to assist students to make an informed decision about potential neighborhoods where to live.
Hence, we will cluster Milan neighborhoods in order to recommend amenities, facilities and buying/rental households prices.

## Data section
Data on Milan neighborhoods and current real estate market are available at:
https://www.agenziaentrate.gov.it/portale/web/guest/schede/fabbricatiterreni/omi/banche-dati/quotazioni-immobiliari/download-gratuito

To explore and target recommended locations we will extract venues using FourSquare API interface.

Merging data on Milan real estates market and venues, we will support the students decision process.

## Methodology section
The Methodology section will describe the main components of our analysis system and is structured in four steps, as follows: 

- Data collection and inspection 
- Data exploration and understanding
- Data preparation and preprocessing 
- Data modeling

Each of these steps is further divided into the activities described below.

#### 1. IMPORTING LIBRARIES

In [1]:
# import libraries

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes  
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# import math library for Nan testing
import math

!conda install -c conda-forge folium=0.5.0 --yes  
import folium # map rendering library

print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.0

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    scikit-learn-0.20.1        |   py36h22eb022_0         5.7 MB
    liblapack-3.8.0            |      11_openblas          10 KB  conda-forge
    liblapacke-3.8.0           |      11_openblas          10 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    libopenblas-0.3.6          |       h5a2b251_2         7.7 MB
    numpy-1.17.3               |   py36h95a1406_0         5.2 MB  conda-forge
    sci

#### 2. NEIGHBORHOODS FETCHING

In [3]:
# milan neighborhoods and real estate data from:
# https://www.agenziaentrate.gov.it/portale/web/guest/schede/fabbricatiterreni/omi/banche-dati/quotazioni-immobiliari/download-gratuito
# csv data imported on github
# read csv files  
neigh_url = 'https://raw.githubusercontent.com/lrusso69/CourseraCapstoneTheBattleOfNeighborhoods/master/milan%20neighborhoods.csv'
neighborhoods=pd.read_csv(neigh_url, sep=';')
neighborhoods.head()

,Zona,Zona_Descr,Zona_sint
0,B12,"DUOMO, SANBABILA, MONTENAPOLEONE, MISSORI, CAI...",DUOMO
1,B13,"UNIVERSITA STATALE, SAN LORENZO",UNIVERSITA STATALE
2,B15,BRERA,BRERA
3,B16,"SANT`AMBROGIO, CADORNA, VIA DANTE",SANT`AMBROGIO
4,B17,"PARCO SEMPIONE, ARCO DELLA PACE, CORSO MAGENTA",PARCO SEMPIONE


#### 3. RENAME NEIGHBORHOODS COLUMNS

In [4]:
# meaningful names
neighborhoods.rename(columns={'Zona':'idBorough',
                              'Zona_Descr':'Borough', 
                              'Zona_sint':'Neighborhood'}, 
                     inplace=True)
neighborhoods.head()

,idBorough,Borough,Neighborhood
0,B12,"DUOMO, SANBABILA, MONTENAPOLEONE, MISSORI, CAI...",DUOMO
1,B13,"UNIVERSITA STATALE, SAN LORENZO",UNIVERSITA STATALE
2,B15,BRERA,BRERA
3,B16,"SANT`AMBROGIO, CADORNA, VIA DANTE",SANT`AMBROGIO
4,B17,"PARCO SEMPIONE, ARCO DELLA PACE, CORSO MAGENTA",PARCO SEMPIONE


#### 4. GETTING COORDINATES OF THE NEIGHBORHOODS 

In [5]:
neighborhoods['Latitude']=''
neighborhoods['Longitude']=''
geolocator = Nominatim(user_agent="milan_explorer")
for i in range(len(neighborhoods)):
    address = neighborhoods['Neighborhood'][i] + ', Milan, Italy'
    print(address)
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    neighborhoods['Latitude'][i]=latitude
    neighborhoods['Longitude'][i]=longitude
neighborhoods

DUOMO, Milan, Italy
UNIVERSITA STATALE, Milan, Italy
BRERA, Milan, Italy
SANT`AMBROGIO, Milan, Italy
PARCO SEMPIONE, Milan, Italy
TURATI, Milan, Italy
PORTA VITTORIA, Milan, Italy
PORTA VIGENTINA, Milan, Italy
PORTA TICINESE, Milan, Italy
CORSO BUENOS AIRES, Milan, Italy
CITY LIFE, Milan, Italy
PORTA NUOVA, Milan, Italy
CENTRALE FS, Milan, Italy
CENISIO, Milan, Italy
SEMPIONE, Milan, Italy
SOLARI, Milan, Italy
TABACCHI, Milan, Italy
CORSO CONCORDIA, Milan, Italy
PARCO LAMBRO, Milan, Italy
PIOLA, Milan, Italy
LAMBRATE, Milan, Italy
FORLANINI, Milan, Italy
LONGANESI, Milan, Italy
VIGENTINO, Milan, Italy
ORTLES, Milan, Italy
BARONA, Milan, Italy
SEGESTA, Milan, Italy
LORENTEGGIO, Milan, Italy
CAPRILLI, Milan, Italy
MUSOCCO, Milan, Italy
BOVISA, Milan, Italy
BOVISASCA, Milan, Italy
NIGUARDA, Milan, Italy
SARCA, Milan, Italy
CRESCENZAGO, Milan, Italy
MAGGIOLINA, Milan, Italy
BAGGIO, Milan, Italy
GALLARATESE, Milan, Italy
GRATOSOGLIO, Milan, Italy
QUARTO OGGIARO, Milan, Italy


,idBorough,Borough,Neighborhood,Latitude,Longitude
0,B12,"DUOMO, SANBABILA, MONTENAPOLEONE, MISSORI, CAI...",DUOMO,45.4641,9.18855
1,B13,"UNIVERSITA STATALE, SAN LORENZO",UNIVERSITA STATALE,45.4619,9.17706
2,B15,BRERA,BRERA,45.4715,9.18774
3,B16,"SANT`AMBROGIO, CADORNA, VIA DANTE",SANT`AMBROGIO,45.4614,9.17292
4,B17,"PARCO SEMPIONE, ARCO DELLA PACE, CORSO MAGENTA",PARCO SEMPIONE,45.473,9.17697
5,B18,"TURATI, MOSCOVA, CORSO VENEZIA",TURATI,45.475,9.19472
6,B19,"VENEZIA, PORTA VITTORIA, PORTA ROMANA",PORTA VITTORIA,45.4623,9.20958
7,B20,"PORTA VIGENTINA, PORTA ROMANA",PORTA VIGENTINA,45.4514,9.19645
8,B21,"PORTA TICINESE, PORTA GENOVA, VIA SAN VITTORE",PORTA TICINESE,45.4525,9.18021
9,C12,"PISANI, BUENOS AIRES, REGINA GIOVANNA",CORSO BUENOS AIRES,45.4796,9.21023


#### 5. GETTING MILAN COORDINATES

In [6]:
address = 'Milan'
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Milan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Milan are 45.4668, 9.1905.


#### 6. MILAN NEIGHBORHOODS MAP

In [7]:
# create map of Milan using latitude and longitude values
map_milan = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, idBorough, Neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Neighborhood'], neighborhoods['idBorough']):
    label = '{}, {}'.format(idBorough, Neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_milan)  
    
map_milan

#### 7. FOURSQUARE CREDENTIALS AND VERSION

In [8]:
CLIENT_ID = 'V5CFEX31RYRPE45J3TYGDLRXXWYLW3V2STARTORV2KPXGXMK' # my Foursquare ID
CLIENT_SECRET = 'AK1KKXWDRPMHHD3CTQJOBRNSLZV5FJPXQEQH3TJYXDJ3U4QT' # my Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('My credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

My credentials:
CLIENT_ID: V5CFEX31RYRPE45J3TYGDLRXXWYLW3V2STARTORV2KPXGXMK
CLIENT_SECRET:AK1KKXWDRPMHHD3CTQJOBRNSLZV5FJPXQEQH3TJYXDJ3U4QT


#### 8. EXPLORE DUOMO NEIGHBORHOOD IN MILAN

In [9]:
neighborhoods.loc[0, 'Neighborhood']

'DUOMO'

In [10]:
neighborhood_latitude = neighborhoods.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = neighborhoods.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = neighborhoods.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of DUOMO are 45.4641385, 9.1885548.


#### 9. GET TOP 100 VENUES IN DUOMO NEIGHBORHOOD WITHIN A RADIUS OF 500 MT

In [11]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=V5CFEX31RYRPE45J3TYGDLRXXWYLW3V2STARTORV2KPXGXMK&client_secret=AK1KKXWDRPMHHD3CTQJOBRNSLZV5FJPXQEQH3TJYXDJ3U4QT&v=20180605&ll=45.4641385,9.1885548&radius=500&limit=100'

In [12]:
# Send the GET request and examine the results
results = requests.get(url).json()

In [13]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [14]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Piazza del Duomo,Plaza,45.464190,9.189527
1,Galleria Vittorio Emanuele II,Monument / Landmark,45.465577,9.190024
2,Terrazze del Duomo,Scenic Lookout,45.464207,9.191075
3,Park Hyatt Milan,Hotel,45.465532,9.188911
4,Room Mate Giulia Hotel,Hotel,45.465250,9.189396


In [15]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


#### 10. FUNCTION TO GET VENUES FOR ALL THE MILAN NEIGHBORHOODS 

In [16]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### 11. RUN THE FUNCTION AND CREATE A NEW DATAFRAME 'MILAN VENUES'

In [17]:
milan_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                 )

DUOMO
UNIVERSITA STATALE
BRERA
SANT`AMBROGIO
PARCO SEMPIONE
TURATI
PORTA VITTORIA
PORTA VIGENTINA
PORTA TICINESE
CORSO BUENOS AIRES
CITY LIFE
PORTA NUOVA
CENTRALE FS
CENISIO
SEMPIONE
SOLARI
TABACCHI
CORSO CONCORDIA
PARCO LAMBRO
PIOLA
LAMBRATE
FORLANINI
LONGANESI
VIGENTINO
ORTLES
BARONA
SEGESTA
LORENTEGGIO
CAPRILLI
MUSOCCO
BOVISA
BOVISASCA
NIGUARDA
SARCA
CRESCENZAGO
MAGGIOLINA
BAGGIO
GALLARATESE
GRATOSOGLIO
QUARTO OGGIARO


In [18]:
print(milan_venues.shape)
milan_venues.head()

(1623, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,DUOMO,45.464138,9.188555,Piazza del Duomo,45.464190,9.189527,Plaza
1,DUOMO,45.464138,9.188555,Galleria Vittorio Emanuele II,45.465577,9.190024,Monument / Landmark
2,DUOMO,45.464138,9.188555,Terrazze del Duomo,45.464207,9.191075,Scenic Lookout
3,DUOMO,45.464138,9.188555,Park Hyatt Milan,45.465532,9.188911,Hotel
4,DUOMO,45.464138,9.188555,Room Mate Giulia Hotel,45.465250,9.189396,Hotel


#### 12. VENUES RETURNED IN MILAN 

In [19]:
milan_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
BAGGIO,9,9,9,9,9,9
BARONA,16,16,16,16,16,16
BOVISA,29,29,29,29,29,29
BOVISASCA,8,8,8,8,8,8
BRERA,100,100,100,100,100,100
CAPRILLI,9,9,9,9,9,9
CENISIO,35,35,35,35,35,35
CENTRALE FS,100,100,100,100,100,100
CITY LIFE,100,100,100,100,100,100


#### 13. VENUES CATEGORIES AND NEIGHBORHOOD ONE HOT ENCODING

In [20]:
print('There are {} uniques categories.'.format(len(milan_venues['Venue Category'].unique())))

There are 210 uniques categories.


In [21]:
# one hot encoding
milan_onehot = pd.get_dummies(milan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
milan_onehot['Neighborhood'] = milan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [milan_onehot.columns[-1]] + list(milan_onehot.columns[:-1])
milan_onehot = milan_onehot[fixed_columns]

milan_onehot.head()

,Neighborhood,Accessories Store,Adult Education Center,African Restaurant,Airport Service,American Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Ballroom,Bar,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bistro,Board Shop,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Burger Joint,Bus Station,Bus Stop,Café,Camera Store,Candy Store,Caribbean Restaurant,Casino,Castle,Chinese Restaurant,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,Comedy Club,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Creperie,Cupcake Shop,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doctor's Office,Dog Run,Doner Restaurant,Electronics Store,Emilia Restaurant,Empanada Restaurant,Event Space,Fabric Shop,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,Fire Station,Fish & Chips Shop,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Gastropub,Gay Bar,Gift Shop,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Theater,Italian Restaurant,Japanese Restaurant,Jewelry Store,Karaoke Bar,Kebab Restaurant,Kitchen Supply Store,Korean Restaurant,Light Rail Station,Lingerie Store,Lombard Restaurant,Lounge,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nightclub,Noodle House,Opera House,Other Nightlife,Outdoor Sculpture,Outdoors & Recreation,Outlet Store,Palace,Paper / Office Supplies Store,Park,Pastry Shop,Peruvian Restaurant,Pet Café,Pharmacy,Piadineria,Pizza Place,Platform,Playground,Plaza,Pool,Pub,Public Art,Puglia Restaurant,Racecourse,Ramen Restaurant,Record Shop,Restaurant,River,Rock Club,Roman Restaurant,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Sicilian Restaurant,Smoke Shop,Snack Place,Soccer Field,Social Club,South Tyrolean Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Stationery Store,Steakhouse,Street Art,Student Center,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tapas Restaurant,Tea Room,Tennis Stadium,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Toy / Game Store,Train Station,Tram Station,Trattoria/Osteria,Tunnel,Tuscan Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,DUOMO,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,DUOMO,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,DUOMO,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [22]:
milan_onehot.shape

(1623, 211)

#### 14. Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [23]:
milan_grouped = milan_onehot.groupby('Neighborhood').mean().reset_index()
milan_grouped.head()

,Neighborhood,Accessories Store,Adult Education Center,African Restaurant,Airport Service,American Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Ballroom,Bar,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bistro,Board Shop,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Burger Joint,Bus Station,Bus Stop,Café,Camera Store,Candy Store,Caribbean Restaurant,Casino,Castle,Chinese Restaurant,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,Comedy Club,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Creperie,Cupcake Shop,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doctor's Office,Dog Run,Doner Restaurant,Electronics Store,Emilia Restaurant,Empanada Restaurant,Event Space,Fabric Shop,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,Fire Station,Fish & Chips Shop,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Gastropub,Gay Bar,Gift Shop,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Theater,Italian Restaurant,Japanese Restaurant,Jewelry Store,Karaoke Bar,Kebab Restaurant,Kitchen Supply Store,Korean Restaurant,Light Rail Station,Lingerie Store,Lombard Restaurant,Lounge,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nightclub,Noodle House,Opera House,Other Nightlife,Outdoor Sculpture,Outdoors & Recreation,Outlet Store,Palace,Paper / Office Supplies Store,Park,Pastry Shop,Peruvian Restaurant,Pet Café,Pharmacy,Piadineria,Pizza Place,Platform,Playground,Plaza,Pool,Pub,Public Art,Puglia Restaurant,Racecourse,Ramen Restaurant,Record Shop,Restaurant,River,Rock Club,Roman Restaurant,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Sicilian Restaurant,Smoke Shop,Snack Place,Soccer Field,Social Club,South Tyrolean Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Stationery Store,Steakhouse,Street Art,Student Center,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tapas Restaurant,Tea Room,Tennis Stadium,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Toy / Game Store,Train Station,Tram Station,Trattoria/Osteria,Tunnel,Tuscan Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,BAGGIO,0.00,0.0,0.0,0.0,0.0,0.0,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.000000,0.000000,0.111111,0.0000,0.0,0.00,0.0,0.0,0.0,0.0000,0.000000,0.00,0.00,0.0,0.00,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.222222,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.00,0.00,0.00,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0000,0.0,0.0,0.0000,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.000000,0.0,0.0,0.222222,0.111111,0.00,0.00,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0000,0.00,0.0000,0.0,0.0,0.0,0.0,0.0,0.00,0.000000,0.000,0.00,0.0,0.0,0.0,0.000000,0.111111,0.000000,0.0,0.111111,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0000,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.000,0.000000,0.00,0.0,0.000,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.000000,0.0,0.0,0.111111,0.00,0.0,0.00,0.00,0.0000,0.0,0.00,0.0,0

In [24]:
milan_grouped.shape

(40, 211)

#### 15. Print each neighborhood along with the top 5 most common venues

In [25]:
num_top_venues = 5

for hood in milan_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = milan_grouped[milan_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----BAGGIO----
                 venue  freq
0   Italian Restaurant  0.22
1                 Café  0.22
2  Japanese Restaurant  0.11
3          Supermarket  0.11
4                Plaza  0.11


----BARONA----
                 venue  freq
0                 Café  0.12
1               Bakery  0.06
2            Gastropub  0.06
3  Fried Chicken Joint  0.06
4         Noodle House  0.06


----BOVISA----
                venue  freq
0                Café  0.14
1  Italian Restaurant  0.10
2            Platform  0.10
3          Piadineria  0.07
4            Ballroom  0.03


----BOVISASCA----
                venue  freq
0          Restaurant  0.12
1      Shopping Plaza  0.12
2  Italian Restaurant  0.12
3         Art Gallery  0.12
4         Supermarket  0.12


----BRERA----
                venue  freq
0  Italian Restaurant  0.19
1               Hotel  0.08
2      Ice Cream Shop  0.05
3            Boutique  0.04
4                Café  0.04


----CAPRILLI----
               venue  freq
0          Nightc

#### 16. VENUES SORTING

In [26]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [27]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = milan_grouped['Neighborhood']

for ind in np.arange(milan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(milan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,BAGGIO,Italian Restaurant,Café,Bar,Japanese Restaurant,Plaza,Pizza Place,Supermarket,Flea Market,Fish & Chips Shop,Fire Station
1,BARONA,Café,Bar,Bakery,Noodle House,Fried Chicken Joint,Board Shop,Gastropub,Supermarket,Bed & Breakfast,Tennis Stadium
2,BOVISA,Café,Italian Restaurant,Platform,Piadineria,Ballroom,Creperie,Pizza Place,College Cafeteria,Restaurant,Paper / Office Supplies Store
3,BOVISASCA,Italian Restaurant,Restaurant,Art Gallery,Soccer Field,Supermarket,Chinese Restaurant,Shopping Plaza,Park,Emilia Restaurant,Fire Station
4,BRERA,Italian Restaurant,Hotel,Ice Cream Shop,Café,Pizza Place,Boutique,Plaza,Sandwich Place,Theater,Japanese Restaurant


In [28]:
neighborhoods_venues_sorted.shape

(40, 11)

#### 17. REAL ESTATE VALUES FETCHING

In [29]:
# milan neighborhoods and real estate data from:
# https://www.agenziaentrate.gov.it/portale/web/guest/schede/fabbricatiterreni/omi/banche-dati/quotazioni-immobiliari/download-gratuito
# csv data imported on github
re_url = 'https://raw.githubusercontent.com/lrusso69/CourseraCapstoneTheBattleOfNeighborhoods/master/milan%20neighborhoods%20real%20estates%20values.csv'
real_estate=pd.read_csv(re_url, sep=';')
real_estate.head()

,Fascia,Zona,Zona_Descr,Descr_Tipologia,Stato,Compr_min,Compr_max,Loc_min,Loc_max
0,B,B12,"DUOMO, SANBABILA, MONTENAPOLEONE, MISSORI, CAI...",Abitazioni civili,OTTIMO,9100,12300,29.0,37.0
1,B,B12,"DUOMO, SANBABILA, MONTENAPOLEONE, MISSORI, CAI...",Abitazioni civili,NORMALE,7400,8900,23.0,29.0
2,B,B12,"DUOMO, SANBABILA, MONTENAPOLEONE, MISSORI, CAI...",Abitazioni di tipo economico,NORMALE,6400,7700,18.5,22.0
3,B,B12,"DUOMO, SANBABILA, MONTENAPOLEONE, MISSORI, CAI...",Abitazioni di tipo economico,OTTIMO,7800,8800,24.0,30.0
4,B,B12,"DUOMO, SANBABILA, MONTENAPOLEONE, MISSORI, CAI...",Abitazioni signorili,OTTIMO,11300,14300,38.5,46.0


#### 18. REAL ESTATE DATAFRAME TRANSFORMATION

In [30]:
# columns renaming with meaningful names
real_estate.rename(columns={  'Fascia':'idGroup',
                              'Zona':'idBorough',
                              'Zona_Descr':'Borough', 
                              'Descr_Tipologia':'Property',
                              'Stato':'Status',
                              'Compr_min':'Min_price',
                              'Compr_max':'Max_price',
                              'Loc_min':'Min_rent',
                              'Loc_max':'Max_rent'}, 
                     inplace=True)
real_estate.head()

,idGroup,idBorough,Borough,Property,Status,Min_price,Max_price,Min_rent,Max_rent
0,B,B12,"DUOMO, SANBABILA, MONTENAPOLEONE, MISSORI, CAI...",Abitazioni civili,OTTIMO,9100,12300,29.0,37.0
1,B,B12,"DUOMO, SANBABILA, MONTENAPOLEONE, MISSORI, CAI...",Abitazioni civili,NORMALE,7400,8900,23.0,29.0
2,B,B12,"DUOMO, SANBABILA, MONTENAPOLEONE, MISSORI, CAI...",Abitazioni di tipo economico,NORMALE,6400,7700,18.5,22.0
3,B,B12,"DUOMO, SANBABILA, MONTENAPOLEONE, MISSORI, CAI...",Abitazioni di tipo economico,OTTIMO,7800,8800,24.0,30.0
4,B,B12,"DUOMO, SANBABILA, MONTENAPOLEONE, MISSORI, CAI...",Abitazioni signorili,OTTIMO,11300,14300,38.5,46.0


In [31]:
# mean price computation and max/min prices dropping
real_estate['Avg_price'] = real_estate[['Min_price','Max_price']].mean(axis=1)
real_estate['Avg_rent'] = real_estate[['Min_rent','Max_rent']].mean(axis=1)
real_estate = real_estate.drop(columns=['Min_price', 'Max_price','Min_rent', 'Max_rent'], axis=1)
real_estate.head()

,idGroup,idBorough,Borough,Property,Status,Avg_price,Avg_rent
0,B,B12,"DUOMO, SANBABILA, MONTENAPOLEONE, MISSORI, CAI...",Abitazioni civili,OTTIMO,10700.0,33.00
1,B,B12,"DUOMO, SANBABILA, MONTENAPOLEONE, MISSORI, CAI...",Abitazioni civili,NORMALE,8150.0,26.00
2,B,B12,"DUOMO, SANBABILA, MONTENAPOLEONE, MISSORI, CAI...",Abitazioni di tipo economico,NORMALE,7050.0,20.25
3,B,B12,"DUOMO, SANBABILA, MONTENAPOLEONE, MISSORI, CAI...",Abitazioni di tipo economico,OTTIMO,8300.0,27.00
4,B,B12,"DUOMO, SANBABILA, MONTENAPOLEONE, MISSORI, CAI...",Abitazioni signorili,OTTIMO,12800.0,42.25


In [32]:
real_estate.shape

(482, 7)

In [33]:
# deleting rows related to buildings which are not households 
rows_to_delete = ['Box', 'Uffici', 'Laboratori', 'Uffici strutturati', 'Laboratori', 
                  'Magazzini', 'Negozi', 'Capannoni tipici', 'Capannoni industriali', 'Ville e Villini']
real_estate = real_estate[~real_estate['Property'].isin(rows_to_delete)]
real_estate = real_estate[~real_estate['Status'].isin(['NORMALE'])]
real_estate = real_estate.drop(columns=['Status'], axis=1)
real_estate = real_estate.reset_index(drop=True)
real_estate.head()

,idGroup,idBorough,Borough,Property,Avg_price,Avg_rent
0,B,B12,"DUOMO, SANBABILA, MONTENAPOLEONE, MISSORI, CAI...",Abitazioni civili,10700.0,33.00
1,B,B12,"DUOMO, SANBABILA, MONTENAPOLEONE, MISSORI, CAI...",Abitazioni di tipo economico,8300.0,27.00
2,B,B12,"DUOMO, SANBABILA, MONTENAPOLEONE, MISSORI, CAI...",Abitazioni signorili,12800.0,42.25
3,B,B13,"UNIVERSITA STATALE, SAN LORENZO",Abitazioni civili,7550.0,23.40
4,B,B13,"UNIVERSITA STATALE, SAN LORENZO",Abitazioni di tipo economico,5750.0,18.15


In [34]:
real_estate.shape

(87, 6)

In [35]:
# new real estate dataframe
columns_names = ['idBorough', 'Low_Household_price', 'Medium_Household_price', 'High_Household_price', 
                 'Low_Household_rent', 'Medium_Household_rent', 'High_Household_rent']
new_real_estate=pd.DataFrame(columns=columns_names)
new_real_estate

,idBorough,Low_Household_price,Medium_Household_price,High_Household_price,Low_Household_rent,Medium_Household_rent,High_Household_rent


In [36]:
medium_Household_price = low_Household_price = high_Household_price = 0
medium_Household_rent = low_Household_rent = high_Household_rent = 0

for i in range(len(real_estate)):
    
    idBorough = real_estate['idBorough'][i]
    
    if real_estate['Property'][i] == 'Abitazioni civili':
        medium_Household_price = real_estate['Avg_price'][i]
        medium_Household_rent = real_estate['Avg_rent'][i]
             
    if real_estate['Property'][i] == 'Abitazioni di tipo economico':
        low_Household_price = real_estate['Avg_price'][i]
        low_Household_rent = real_estate['Avg_rent'][i]
        
    if real_estate['Property'][i] == 'Abitazioni signorili':
        high_Household_price = real_estate['Avg_price'][i] 
        high_Household_rent = real_estate['Avg_rent'][i]
      
    if ((i>0 and real_estate['idBorough'][i] != real_estate['idBorough'][i-1]) or i==0):       
        new_real_estate = new_real_estate.append({ 'idBorough': idBorough,
                                                   'Low_Household_price': low_Household_price,
                                                   'Medium_Household_price': medium_Household_price,
                                                   'High_Household_price': high_Household_price,
                                                   'Low_Household_rent': low_Household_price,
                                                   'Medium_Household_rent': medium_Household_rent,
                                                   'High_Household_rent': high_Household_price}, ignore_index=True)
                
    else:
        new_real_estate['Low_Household_price'][new_real_estate.index[-1]] = low_Household_price
        new_real_estate['Medium_Household_price'][new_real_estate.index[-1]] = medium_Household_price
        new_real_estate['High_Household_price'][new_real_estate.index[-1]] = high_Household_price
        new_real_estate['Low_Household_rent'][new_real_estate.index[-1]] = low_Household_rent
        new_real_estate['Medium_Household_rent'][new_real_estate.index[-1]] = medium_Household_rent
        new_real_estate['High_Household_rent'][new_real_estate.index[-1]] = high_Household_rent
                
new_real_estate.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/jupyterlab/conda/envs/python/lib/pyth

,idBorough,Low_Household_price,Medium_Household_price,High_Household_price,Low_Household_rent,Medium_Household_rent,High_Household_rent
0,B12,8300,10700.0,12800,27,33.00,42.25
1,B13,5750,7550.0,8550,18.15,23.40,26.45
2,B15,7350,9150.0,11200,22.55,25.30,35.4
3,B16,6850,8500.0,10000,19.8,23.60,31
4,B17,4550,6900.0,9250,15.15,20.85,26.1


#### 19. VENUES AND REAL ESTATE NEIGHBORHOODS DATAFRAMES JOINING

In [37]:
new_real_estate = pd.merge(new_real_estate, neighborhoods, on='idBorough')
new_real_estate.head()

,idBorough,Low_Household_price,Medium_Household_price,High_Household_price,Low_Household_rent,Medium_Household_rent,High_Household_rent,Borough,Neighborhood,Latitude,Longitude
0,B12,8300,10700.0,12800,27,33.00,42.25,"DUOMO, SANBABILA, MONTENAPOLEONE, MISSORI, CAI...",DUOMO,45.4641,9.18855
1,B13,5750,7550.0,8550,18.15,23.40,26.45,"UNIVERSITA STATALE, SAN LORENZO",UNIVERSITA STATALE,45.4619,9.17706
2,B15,7350,9150.0,11200,22.55,25.30,35.4,BRERA,BRERA,45.4715,9.18774
3,B16,6850,8500.0,10000,19.8,23.60,31,"SANT`AMBROGIO, CADORNA, VIA DANTE",SANT`AMBROGIO,45.4614,9.17292
4,B17,4550,6900.0,9250,15.15,20.85,26.1,"PARCO SEMPIONE, ARCO DELLA PACE, CORSO MAGENTA",PARCO SEMPIONE,45.473,9.17697


In [38]:
new_real_estate = new_real_estate.drop(columns=['idBorough', 'Borough','Latitude', 'Longitude'], axis=1)
new_real_estate.head()

,Low_Household_price,Medium_Household_price,High_Household_price,Low_Household_rent,Medium_Household_rent,High_Household_rent,Neighborhood
0,8300,10700.0,12800,27,33.00,42.25,DUOMO
1,5750,7550.0,8550,18.15,23.40,26.45,UNIVERSITA STATALE
2,7350,9150.0,11200,22.55,25.30,35.4,BRERA
3,6850,8500.0,10000,19.8,23.60,31,SANT`AMBROGIO
4,4550,6900.0,9250,15.15,20.85,26.1,PARCO SEMPIONE


In [39]:
milan_to_cluster = pd.merge(milan_grouped, new_real_estate, on='Neighborhood')
milan_to_cluster.head()

,Neighborhood,Accessories Store,Adult Education Center,African Restaurant,Airport Service,American Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Ballroom,Bar,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bistro,Board Shop,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Burger Joint,Bus Station,Bus Stop,Café,Camera Store,Candy Store,Caribbean Restaurant,Casino,Castle,Chinese Restaurant,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,Comedy Club,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Creperie,Cupcake Shop,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doctor's Office,Dog Run,Doner Restaurant,Electronics Store,Emilia Restaurant,Empanada Restaurant,Event Space,Fabric Shop,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,Fire Station,Fish & Chips Shop,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Gastropub,Gay Bar,Gift Shop,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Theater,Italian Restaurant,Japanese Restaurant,Jewelry Store,Karaoke Bar,Kebab Restaurant,Kitchen Supply Store,Korean Restaurant,Light Rail Station,Lingerie Store,Lombard Restaurant,Lounge,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nightclub,Noodle House,Opera House,Other Nightlife,Outdoor Sculpture,Outdoors & Recreation,Outlet Store,Palace,Paper / Office Supplies Store,Park,Pastry Shop,Peruvian Restaurant,Pet Café,Pharmacy,Piadineria,Pizza Place,Platform,Playground,Plaza,Pool,Pub,Public Art,Puglia Restaurant,Racecourse,Ramen Restaurant,Record Shop,Restaurant,River,Rock Club,Roman Restaurant,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Sicilian Restaurant,Smoke Shop,Snack Place,Soccer Field,Social Club,South Tyrolean Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Stationery Store,Steakhouse,Street Art,Student Center,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tapas Restaurant,Tea Room,Tennis Stadium,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Toy / Game Store,Train Station,Tram Station,Trattoria/Osteria,Tunnel,Tuscan Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Low_Household_price,Medium_Household_price,High_Household_price,Low_Household_rent,Medium_Household_rent,High_Household_rent
0,BAGGIO,0.00,0.0,0.0,0.0,0.0,0.0,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.000000,0.000000,0.111111,0.0000,0.0,0.00,0.0,0.0,0.0,0.0000,0.000000,0.00,0.00,0.0,0.00,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.222222,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.00,0.00,0.00,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0000,0.0,0.0,0.0000,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.000000,0.0,0.0,0.222222,0.111111,0.00,0.00,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0000,0.00,0.0000,0.0,0.0,0.0,0.0,0.0,0.00,0.000000,0.000,0.00,0.0,0.0,0.0,0.000000,0.111111,0.000000,0.0,0.111111,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0000,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.000,0.00000

#### 20. Run *k*-means to cluster the neighborhoods into 5 clusters

In [40]:
# set number of clusters
kclusters = 5

milan_to_cluster = milan_to_cluster.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(milan_to_cluster)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([0, 0, 0, 0, 1, 0, 3, 3, 2, 2, 4, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 2, 0, 1, 2, 2, 2, 0, 1, 0, 0, 4, 4, 4, 2, 2, 0], dtype=int32)

#### 21. Create a new dataframe with clusters and top 10 venues for each neighborhood

In [41]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

milan_merged = neighborhoods

milan_merged = milan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
milan_merged = milan_merged.join(new_real_estate.set_index('Neighborhood'), on='Neighborhood')

milan_merged.head()

,idBorough,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Low_Household_price,Medium_Household_price,High_Household_price,Low_Household_rent,Medium_Household_rent,High_Household_rent
0,B12,"DUOMO, SANBABILA, MONTENAPOLEONE, MISSORI, CAI...",DUOMO,45.4641,9.18855,1,Italian Restaurant,Plaza,Hotel,Ice Cream Shop,Café,Pizza Place,Boutique,Gourmet Shop,Monument / Landmark,Bookstore,8300,10700.0,12800,27,33.00,42.25
1,B13,"UNIVERSITA STATALE, SAN LORENZO",UNIVERSITA STATALE,45.4619,9.17706,2,Italian Restaurant,Café,Pizza Place,Plaza,Science Museum,Thrift / Vintage Store,Pub,Restaurant,Dessert Shop,Ice Cream Shop,5750,7550.0,8550,18.15,23.40,26.45
2,B15,BRERA,BRERA,45.4715,9.18774,1,Italian Restaurant,Hotel,Ice Cream Shop,Café,Pizza Place,Boutique,Plaza,Sandwich Place,Theater,Japanese Restaurant,7350,9150.0,11200,22.55,25.30,35.4
3,B16,"SANT`AMBROGIO, CADORNA, VIA DANTE",SANT`AMBROGIO,45.4614,9.17292,1,Pizza Place,Italian Restaurant,Café,Pub,Plaza,Historic Site,Japanese Restaurant,Gym,Restaurant,Science Museum,6850,8500.0,10000,19.8,23.60,31
4,B17,"PARCO SEMPIONE, ARCO DELLA PACE, CORSO MAGENTA",PARCO SEMPIONE,45.473,9.17697,2,Theater,Café,Nightclub,Italian Restaurant,Museum,Bookstore,Monument / Landmark,Smoke Shop,Burger Joint,Scenic Lookout,4550,6900.0,9250,15.15,20.85,26.1


#### 22. Visualize the resulting clusters

In [42]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(milan_merged['Latitude'], milan_merged['Longitude'], milan_merged['Neighborhood'], milan_merged['Cluster Labels']):
    if math.isnan(cluster) == False: # skip NaN cases
        label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
        folium.CircleMarker(
            [lat, lon],
            radius=5,
            popup=label,
            color=rainbow[int(cluster)-1],
            fill=True,
            fill_color=rainbow[int(cluster)],
            fill_opacity=0.7).add_to(map_clusters)

map_clusters

#### 23. EXPLORATION OF EACH CLUSTER

In [43]:
milan_merged.loc[milan_merged['Cluster Labels'] == 0, milan_merged.
                 columns[[1] + list(range(5, milan_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Low_Household_price,Medium_Household_price,High_Household_price,Low_Household_rent,Medium_Household_rent,High_Household_rent
18,"PARCO LAMBRO, FELTRE, UDINE",0,Fire Station,Yoga Studio,Convention Center,Food & Drink Shop,Food,Flea Market,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant,1975,3100.0,6450,7.3,10.30,21.35
19,"PIOLA, ARGONNE, CORSICA",0,Pizza Place,Hotel,Italian Restaurant,Bar,Plaza,Café,Restaurant,Ice Cream Shop,Food Truck,Gym,2350,3700.0,6450,7.8,11.50,21.35
20,"LAMBRATE, RUBATTINO, ROMBON",0,Italian Restaurant,Pizza Place,Café,Plaza,Supermarket,Hotel,Indian Restaurant,Pharmacy,Platform,Comedy Club,1975,2750.0,6450,5.75,8.40,21.35
21,"FORLANINI, MECENATE, ORTOMERCATO, SANTA GIULIA",0,Airport Service,Scenic Lookout,Yoga Studio,Emilia Restaurant,Food,Flea Market,Fish & Chips Shop,Fire Station,Filipino Restaurant,Fast Food Restaurant,1675,2925.0,6450,5.35,9.35,21.35
22,"TITO LIVIO, TERTULLIANO, LONGANESI",0,Italian Restaurant,Pizza Place,Japanese Restaurant,Hotel,Health Food Store,Café,Supermarket,Cocktail Bar,Asian Restaurant,Dessert Shop,2050,3575.0,6450,6.55,10.25,21.35
23,"MAROCCHETTI, VIGENTINO, CHIESA ROSSA",0,Italian Restaurant,Gastropub,Nightclub,Restaurant,Supermarket,Pub,Breakfast Spot,Café,Chinese Restaurant,Rock Club,1975,3025.0,6450,5.3,8.75,21.35
24,"ORTLES, SPADOLINI, BAZZI",0,Italian Restaurant,Pizza Place,Café,Bowling Alley,Japanese Restaurant,Gym,Gastropub,Mexican Restaurant,Flea Market,Nightclub,2250,3300.0,6450,7.7,11.70,21.35
25,"BARONA, FAMAGOSTA, FAENZA",0,Café,Bar,Bakery,Noodle House,Fried Chicken Joint,Board Shop,Gastropub,Supermarket,Bed & Breakfast,Tennis Stadium,2225,3100.0,6450,7.9,11.90,21.35
26,"SEGESTA, ARETUSA, VESPRI SICILIANI",0,Plaza,Sushi Restaurant,Café,Concert Hall,Japanese Restaurant,Rock Club,Food,Emilia Restaurant,Fish & Chips Shop,Fire Station,2350,3425.0,6450,8.35,11.45,21.35
27,"LORENTEGGIO, INGANNI, BISCEGLIE, SAN CARLO B.",0,Italian Restaurant,Supermarket,Pizza Place,Café,Dessert Shop,Park,Food Court,Japanese Restaurant,Historic Site,Electronics Store,1900,2700.0,6450,7,9.90,21.35


In [44]:
milan_merged.loc[milan_merged['Cluster Labels'] == 1, milan_merged.
                 columns[[1] + list(range(5, milan_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Low_Household_price,Medium_Household_price,High_Household_price,Low_Household_rent,Medium_Household_rent,High_Household_rent
0,"DUOMO, SANBABILA, MONTENAPOLEONE, MISSORI, CAI...",1,Italian Restaurant,Plaza,Hotel,Ice Cream Shop,Café,Pizza Place,Boutique,Gourmet Shop,Monument / Landmark,Bookstore,8300,10700.0,12800,27,33.0,42.25
2,BRERA,1,Italian Restaurant,Hotel,Ice Cream Shop,Café,Pizza Place,Boutique,Plaza,Sandwich Place,Theater,Japanese Restaurant,7350,9150.0,11200,22.55,25.3,35.4
3,"SANT`AMBROGIO, CADORNA, VIA DANTE",1,Pizza Place,Italian Restaurant,Café,Pub,Plaza,Historic Site,Japanese Restaurant,Gym,Restaurant,Science Museum,6850,8500.0,10000,19.8,23.6,31
11,PORTA NUOVA,1,Italian Restaurant,Restaurant,Hotel,Pizza Place,Café,Ice Cream Shop,Plaza,Sushi Restaurant,Furniture / Home Store,Piadineria,4600,9950.0,12500,14.6,34.0,42.65


In [45]:
milan_merged.loc[milan_merged['Cluster Labels'] == 2, milan_merged.
                 columns[[1] + list(range(5, milan_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Low_Household_price,Medium_Household_price,High_Household_price,Low_Household_rent,Medium_Household_rent,High_Household_rent
1,"UNIVERSITA STATALE, SAN LORENZO",2,Italian Restaurant,Café,Pizza Place,Plaza,Science Museum,Thrift / Vintage Store,Pub,Restaurant,Dessert Shop,Ice Cream Shop,5750,7550.0,8550,18.15,23.40,26.45
4,"PARCO SEMPIONE, ARCO DELLA PACE, CORSO MAGENTA",2,Theater,Café,Nightclub,Italian Restaurant,Museum,Bookstore,Monument / Landmark,Smoke Shop,Burger Joint,Scenic Lookout,4550,6900.0,9250,15.15,20.85,26.1
5,"TURATI, MOSCOVA, CORSO VENEZIA",2,Hotel,Italian Restaurant,Art Gallery,Japanese Restaurant,Park,Ice Cream Shop,Gym,Plaza,Beer Store,Bistro,5250,7550.0,9250,18.6,23.15,26.1
6,"VENEZIA, PORTA VITTORIA, PORTA ROMANA",2,Italian Restaurant,Pizza Place,Seafood Restaurant,Café,Electronics Store,Hotel,Bakery,Massage Studio,Sandwich Place,Russian Restaurant,5450,7250.0,8750,17.1,22.25,26.35
7,"PORTA VIGENTINA, PORTA ROMANA",2,Italian Restaurant,Wine Bar,Pizza Place,Café,Bar,Pub,Burger Joint,Cocktail Bar,Fried Chicken Joint,Restaurant,4750,6100.0,8750,16.15,22.00,26.35
8,"PORTA TICINESE, PORTA GENOVA, VIA SAN VITTORE",2,Italian Restaurant,Ice Cream Shop,Cocktail Bar,Bar,Hotel,Café,Seafood Restaurant,Boutique,Sushi Restaurant,Plaza,4750,6450.0,8750,13.6,17.60,26.35
9,"PISANI, BUENOS AIRES, REGINA GIOVANNA",2,Italian Restaurant,Pizza Place,Café,Hotel,Dessert Shop,Ice Cream Shop,Japanese Restaurant,Cocktail Bar,Steakhouse,Bar,4600,6000.0,8750,14.6,19.05,26.35
10,CITY LIFE,2,Italian Restaurant,Hotel,Café,Pizza Place,Sandwich Place,Ice Cream Shop,Cocktail Bar,Bistro,Japanese Restaurant,Restaurant,4600,8450.0,10200,14.6,25.50,34.8


In [46]:
milan_merged.loc[milan_merged['Cluster Labels'] == 3, milan_merged.
                 columns[[1] + list(range(5, milan_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Low_Household_price,Medium_Household_price,High_Household_price,Low_Household_rent,Medium_Household_rent,High_Household_rent
12,"STAZIONE CENTRALE, VIALE STELVIO",3,Italian Restaurant,Hotel,Café,Ice Cream Shop,Pizza Place,Japanese Restaurant,Sandwich Place,Chinese Restaurant,Restaurant,Breakfast Spot,2425,3900.0,12500,7.3,10.85,42.65
13,"CENISIO, FARINI, SARPI",3,Italian Restaurant,Hotel,Pizza Place,Café,Seafood Restaurant,Tram Station,Sushi Restaurant,Thai Restaurant,Restaurant,Nightclub,2900,4500.0,12500,9.3,12.50,42.65


In [47]:
milan_merged.loc[milan_merged['Cluster Labels'] == 4, milan_merged.
                 columns[[1] + list(range(5, milan_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Low_Household_price,Medium_Household_price,High_Household_price,Low_Household_rent,Medium_Household_rent,High_Household_rent
14,"SEMPIONE, PAGANO, WASHINGTON",4,Italian Restaurant,Pizza Place,Cocktail Bar,Japanese Restaurant,Ice Cream Shop,Bakery,Mediterranean Restaurant,Dessert Shop,Burger Joint,Sushi Restaurant,3850,5250.0,6450,12.15,14.80,21.35
15,"SOLARI, P.TA GENOVA, ASCANIO SFORZA",4,Café,Boutique,Pizza Place,Burger Joint,Pub,Plaza,Italian Restaurant,Wine Bar,Lounge,Ice Cream Shop,3400,4800.0,6450,9.85,12.75,21.35
16,"TABACCHI, SARFATTI, CREMA",4,Italian Restaurant,Café,Hotel,Restaurant,Plaza,Japanese Restaurant,Pizza Place,Asian Restaurant,Chinese Restaurant,Bistro,3275,4750.0,6450,9.45,13.45,21.35
17,"LIBIA, ,XXII MARZO, INDIPENDENZA",4,Italian Restaurant,Café,Pizza Place,Seafood Restaurant,Hotel,Supermarket,Ice Cream Shop,Coffee Shop,Japanese Restaurant,BBQ Joint,3900,5650.0,6450,11.6,16.75,21.35


## Results and Discussion section
Analyzing the Milan neighborhoods and clusters we have identified, we can recognize three main patterns:

- the first pattern refers to Milan "Old town" and includes cluster 1 and 2; characterized by monuments, restaurants, cafes and boutiques, as well as high purchase and rental prices, these clusters might target students who loves amenities and a cultural life, provided they can afford the high cost of living

- the second pattern refers to Milan residential neighborhoods and includes cluster 3 and 4; characterized by several facilities as well as medium purchase and rental prices, this cluster might target students who want to live in stately residential neighborhoods well connected to the city centre 

- finally, the third pattern refers to Milan suburban districts and includes cluster 0; characterized by ethnic amenities and lower purchase and rental prices, this cluster might target students who loves multicultural neighborhoods and want to pursue a more affordable lifestyle


## Conclusion
To sum up, according to the main financial news firms, the Housing Market in Milan (Italy) has been facing for the last five years a growing trend in the households selling prices as well in the rental ones and a further prices acceleration is expected in 2020 due to the "Brexit", which will probably result in a shift of real estate investments from London to the other UE cities.
In this scenario, many students who will pursue their studies in Milan need to know more about the real estate market, in order to correctly choose the neighborhood where to live according to the available public services and venues and to the households buying or rental prices.
For this purpose, we clustered Milan neighborhoods in order to recommend venues and the current price of real estate where students can make an informed decision.
First, we gathered data on Milan neighborhoods and households prices and then we explored and targeted recommended locations across different venues according to the presence of amenities and essential facilities and real estate prices. 
We accessed data through FourSquare API interface and merged them with the real estate ones extracted from "agenzia delle entrate" web site.
Moreover, after data collection and exploration, we addressed the business problem using the k-means clustering technique, due to its efficiency in terms of computational cost and accuracy.

Finally, we drew the conclusion identifying three main patterns in Milan:

-  "Old town", characterized by monuments, restaurants, cafes and boutiques, as well as high purchase and rental prices, suitable for students who loves amenities and a cultural life

- Residential neighborhoods, characterized by several facilities as well as medium purchase and rental prices, suitable for students who want to live in stately residential neighborhoods well connected to the city centre 

- Suburban districts, characterized by ethnic amenities and lower purchase and rental prices, suitable for students who loves multicultural neighborhoods and want to pursue a more affordable lifestyle